### Import Libraries

In [1]:
import configparser as configparser
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf

## Parsing INI File

### Funtion: To parse INI file

In [2]:
def parse_ini(section: str) -> dict:
    """
    This function parses ini file for configuration details
    :param section: section to read from ini
    :return: Dictionary of config details
    """
    config = dict()
    parser = configparser.ConfigParser()
    parser.read("imdb_database.ini")
    if parser.has_section(section):
        config_items = parser.items(section)
        for item in config_items:
            config[item[0]] = item[1]
    return config

In [3]:
datasets = parse_ini("data")
datasets

{'namebasics': 'name.basics.tsv\\data.tsv',
 'titlebasics': 'title.basics.tsv\\data.tsv',
 'titlecrew': 'title.crew.tsv\\data.tsv',
 'titleprincipals': 'title.principals.tsv\\data.tsv',
 'titleratings': 'title.ratings.tsv\\data.tsv'}

In [4]:
spark = SparkSession.builder.master("local[1]").appName('IMDB').getOrCreate()
spark

In [5]:
df_members = spark.read.options(header='True', inferSchema='True', delimiter='\t').csv(datasets["namebasics"])
df_members.printSchema()
df_members.head(10)

root
 |-- nconst: string (nullable = true)
 |-- primaryName: string (nullable = true)
 |-- birthYear: string (nullable = true)
 |-- deathYear: string (nullable = true)
 |-- primaryProfession: string (nullable = true)
 |-- knownForTitles: string (nullable = true)



[Row(nconst='nm0000001', primaryName='Fred Astaire', birthYear='1899', deathYear='1987', primaryProfession='soundtrack,actor,miscellaneous', knownForTitles='tt0053137,tt0050419,tt0072308,tt0045537'),
 Row(nconst='nm0000002', primaryName='Lauren Bacall', birthYear='1924', deathYear='2014', primaryProfession='actress,soundtrack', knownForTitles='tt0037382,tt0071877,tt0038355,tt0117057'),
 Row(nconst='nm0000003', primaryName='Brigitte Bardot', birthYear='1934', deathYear='\\N', primaryProfession='actress,soundtrack,music_department', knownForTitles='tt0057345,tt0049189,tt0056404,tt0054452'),
 Row(nconst='nm0000004', primaryName='John Belushi', birthYear='1949', deathYear='1982', primaryProfession='actor,soundtrack,writer', knownForTitles='tt0077975,tt0080455,tt0072562,tt0078723'),
 Row(nconst='nm0000005', primaryName='Ingmar Bergman', birthYear='1918', deathYear='2007', primaryProfession='writer,director,actor', knownForTitles='tt0050976,tt0083922,tt0050986,tt0060827'),
 Row(nconst='nm000

In [6]:
df_titles = spark.read.options(header='True', inferSchema='True', delimiter='\t').csv(datasets["titlebasics"])
df_titles = df_titles.where("isAdult == 0")
df_titles.printSchema()
df_titles.head(10)

root
 |-- tconst: string (nullable = true)
 |-- titleType: string (nullable = true)
 |-- primaryTitle: string (nullable = true)
 |-- originalTitle: string (nullable = true)
 |-- isAdult: string (nullable = true)
 |-- startYear: string (nullable = true)
 |-- endYear: string (nullable = true)
 |-- runtimeMinutes: string (nullable = true)
 |-- genres: string (nullable = true)



[Row(tconst='tt0000001', titleType='short', primaryTitle='Carmencita', originalTitle='Carmencita', isAdult='0', startYear='1894', endYear='\\N', runtimeMinutes='1', genres='Documentary,Short'),
 Row(tconst='tt0000002', titleType='short', primaryTitle='Le clown et ses chiens', originalTitle='Le clown et ses chiens', isAdult='0', startYear='1892', endYear='\\N', runtimeMinutes='5', genres='Animation,Short'),
 Row(tconst='tt0000003', titleType='short', primaryTitle='Pauvre Pierrot', originalTitle='Pauvre Pierrot', isAdult='0', startYear='1892', endYear='\\N', runtimeMinutes='4', genres='Animation,Comedy,Romance'),
 Row(tconst='tt0000004', titleType='short', primaryTitle='Un bon bock', originalTitle='Un bon bock', isAdult='0', startYear='1892', endYear='\\N', runtimeMinutes='12', genres='Animation,Short'),
 Row(tconst='tt0000005', titleType='short', primaryTitle='Blacksmith Scene', originalTitle='Blacksmith Scene', isAdult='0', startYear='1893', endYear='\\N', runtimeMinutes='1', genres='C

In [7]:
df_principals = spark.read.options(header='True', inferSchema='True', delimiter='\t').csv(datasets["titleprincipals"])
df_principals.printSchema()
df_principals.head(10)

root
 |-- tconst: string (nullable = true)
 |-- ordering: integer (nullable = true)
 |-- nconst: string (nullable = true)
 |-- category: string (nullable = true)
 |-- job: string (nullable = true)
 |-- characters: string (nullable = true)



[Row(tconst='tt0000001', ordering=1, nconst='nm1588970', category='self', job='\\N', characters='["Self"]'),
 Row(tconst='tt0000001', ordering=2, nconst='nm0005690', category='director', job='\\N', characters='\\N'),
 Row(tconst='tt0000001', ordering=3, nconst='nm0374658', category='cinematographer', job='director of photography', characters='\\N'),
 Row(tconst='tt0000002', ordering=1, nconst='nm0721526', category='director', job='\\N', characters='\\N'),
 Row(tconst='tt0000002', ordering=2, nconst='nm1335271', category='composer', job='\\N', characters='\\N'),
 Row(tconst='tt0000003', ordering=1, nconst='nm0721526', category='director', job='\\N', characters='\\N'),
 Row(tconst='tt0000003', ordering=2, nconst='nm1770680', category='producer', job='producer', characters='\\N'),
 Row(tconst='tt0000003', ordering=3, nconst='nm1335271', category='composer', job='\\N', characters='\\N'),
 Row(tconst='tt0000003', ordering=4, nconst='nm5442200', category='editor', job='\\N', characters='\\N'

### Alive actors whose name starts with “Phi” and did not participate in any movie in 2014.

In [8]:
alive_members = df_members.select("nconst", "primaryName").where("deathYear == '\\\\N' AND primaryName LIKE 'Phi%'")
alive_actors = df_principals.select("nconst", "tconst").where("category IN ('self', 'actor', 'actress')").join(alive_members, 'nconst', 'inner')
movies_not_in_2014 = df_titles.select("tconst")\
    .where("(startyear > 2014 OR (CASE WHEN titleType LIKE 'tv%' THEN (CASE WHEN startyear < 2014 THEN (endyear < 2014) ELSE false END) ELSE false END))")
alive_actors = alive_actors.join(movies_not_in_2014, 'tconst', 'inner').select('primaryName').distinct()
output = alive_actors.collect()
output

[Row(primaryName='Philip Casnoff'),
 Row(primaryName='Philippe Batel'),
 Row(primaryName='Philippe Viotto'),
 Row(primaryName='Philip Bolden'),
 Row(primaryName='Phil Rynda'),
 Row(primaryName='Phillip Chea'),
 Row(primaryName='Phillip Groenweghe'),
 Row(primaryName='Philomena Gabriel'),
 Row(primaryName='Phil Marriott'),
 Row(primaryName='Philippe Simonet'),
 Row(primaryName='Philip Dorrell'),
 Row(primaryName='Phil Tufnell'),
 Row(primaryName='Philipp Schmitz-Eisen'),
 Row(primaryName='Philippe Ross'),
 Row(primaryName='Phil Catalli'),
 Row(primaryName='Phil McRea'),
 Row(primaryName='Philippe Levasseur'),
 Row(primaryName='Philline Goess-Enzenberg'),
 Row(primaryName='Philip Holloway'),
 Row(primaryName='Phil Husbands'),
 Row(primaryName='Phil Rees'),
 Row(primaryName='Philip Donoghue'),
 Row(primaryName='Philip Malzahn'),
 Row(primaryName='Philipa Martin'),
 Row(primaryName='Philip Horst'),
 Row(primaryName='Phillip Studmire'),
 Row(primaryName='Philip Spartis'),
 Row(primaryName='

In [9]:
alive_actors.show(10)

+------------------+
|       primaryName|
+------------------+
|    Philip Casnoff|
|    Philippe Batel|
|   Philippe Viotto|
|     Philip Bolden|
|        Phil Rynda|
|      Phillip Chea|
|Phillip Groenweghe|
| Philomena Gabriel|
|     Phil Marriott|
|  Philippe Simonet|
+------------------+
only showing top 10 rows



### Producers who have produced the most talk shows in 2017 and whose name contains “Gill”.

In [10]:
members = df_members.select("nconst", "primaryName").where(df_members.primaryName.contains("Gill"))
producers = df_principals.select("nconst", "tconst").where(df_principals.category == 'producer').join(members, 'nconst', 'inner')
titles = df_titles.select('tconst', sf.split(df_titles.genres, ',', -1).alias('genres'))
talk_shows = titles.where(df_titles.genres.contains('Talk-Show'))
producers = producers.join(talk_shows, 'tconst', 'inner')\
        .groupBy('nconst')\
                .agg(sf.max(producers.primaryName).alias('name'), sf.count(producers.tconst).alias('numOfShows'))
producers = producers.sort(producers.numOfShows.desc()).select('name','numOfShows')
output = producers.collect()
output

[Row(name='Dominic Gillette', numOfShows=220),
 Row(name='Ryan Gill', numOfShows=81),
 Row(name='Gill Stribling-Wright', numOfShows=52),
 Row(name='Shane Gill', numOfShows=38),
 Row(name='Corinne Gilliard', numOfShows=29),
 Row(name='Rosemary Gill', numOfShows=22),
 Row(name='Tom Gillett', numOfShows=5),
 Row(name='Gill Brown', numOfShows=4),
 Row(name='Marc Gilliatt', numOfShows=2),
 Row(name='Dorothea Gillemot', numOfShows=2),
 Row(name='Marianne Gillgren', numOfShows=1),
 Row(name='Gilles Bérard', numOfShows=1),
 Row(name='Gilliam Parker', numOfShows=1),
 Row(name='Susan Gill', numOfShows=1),
 Row(name='Marilyn Gill', numOfShows=1),
 Row(name='Patrick Gillespie', numOfShows=1),
 Row(name='Gilliam Guthrie', numOfShows=1),
 Row(name='Anne McGill Wilson', numOfShows=1)]

In [11]:
producers.show(10)

+--------------------+----------+
|                name|numOfShows|
+--------------------+----------+
|    Dominic Gillette|       220|
|           Ryan Gill|        81|
|Gill Stribling-Wr...|        52|
|          Shane Gill|        38|
|    Corinne Gilliard|        29|
|       Rosemary Gill|        22|
|         Tom Gillett|         5|
|          Gill Brown|         4|
|       Marc Gilliatt|         2|
|   Dorothea Gillemot|         2|
+--------------------+----------+
only showing top 10 rows



### Alive producers with the greatest number of long-run titles produced (runtime greater than 120 minutes).

In [12]:
alive_members = df_members.select("nconst", "primaryName").where("deathYear == '\\\\N'")
alive_producers = df_principals.select("nconst", "tconst").where(df_principals.category == 'producer').join(alive_members, 'nconst', 'inner')
long_titles = df_titles.select('tconst').where(df_titles.runtimeMinutes > 120)
alive_producers = alive_producers.join(long_titles, 'tconst', 'inner')
alive_producers = alive_producers.groupBy('nconst')\
        .agg(sf.max(alive_producers.primaryName).alias('name'), sf.count(alive_producers.tconst).alias('numOfLongRunTitles'))
alive_producers = alive_producers.sort(alive_producers.numOfLongRunTitles.desc()).select('name','numOfLongRunTitles')
output = alive_producers.collect()
output

[Row(name='Acun Ilicali', numOfLongRunTitles=241),
 Row(name='Maxwell James', numOfLongRunTitles=176),
 Row(name='Wade Baverstock', numOfLongRunTitles=143),
 Row(name='Vince McMahon', numOfLongRunTitles=141),
 Row(name='Christopher Lockey', numOfLongRunTitles=114),
 Row(name='Nick Rylance', numOfLongRunTitles=102),
 Row(name='Efe Irvül', numOfLongRunTitles=102),
 Row(name='Matt Spencer', numOfLongRunTitles=92),
 Row(name='Claire Mooney', numOfLongRunTitles=90),
 Row(name='Fatih Aksoy', numOfLongRunTitles=87),
 Row(name='Yasar Irvül', numOfLongRunTitles=86),
 Row(name='Kyle Shire', numOfLongRunTitles=86),
 Row(name='Bhushan Kumar', numOfLongRunTitles=79),
 Row(name='Kerem Çatay', numOfLongRunTitles=78),
 Row(name='Shrikant Mohta', numOfLongRunTitles=76),
 Row(name='John Michael Flynn', numOfLongRunTitles=74),
 Row(name='Ashley Dixon', numOfLongRunTitles=70),
 Row(name='Isobel Rowland', numOfLongRunTitles=64),
 Row(name='Süreyya Önal', numOfLongRunTitles=64),
 Row(name='Natalie Bakhurst'

In [13]:
alive_producers.show(10)

+------------------+------------------+
|              name|numOfLongRunTitles|
+------------------+------------------+
|      Acun Ilicali|               241|
|     Maxwell James|               176|
|   Wade Baverstock|               143|
|     Vince McMahon|               141|
|Christopher Lockey|               114|
|      Nick Rylance|               102|
|         Efe Irvül|               102|
|      Matt Spencer|                92|
|     Claire Mooney|                90|
|       Fatih Aksoy|                87|
+------------------+------------------+
only showing top 10 rows



### Alive actors who have portrayed Jesus Christ (look for both words independently).

In [14]:
alive_members = df_members.select("nconst", "primaryName").where("deathYear == '\\\\N'")
alive_actors = df_principals.select("nconst", "tconst", "characters").where("category IN ('self', 'actor', 'actress')")\
    .where(df_principals.characters.contains('"Jesus"') | df_principals.characters.contains('"Christ"') | df_principals.characters.contains('"Jesus Christ"'))\
        .join(alive_members, 'nconst', 'inner').select('primaryName', 'characters')
output = alive_actors.collect()
output

[Row(primaryName='Ron Ferguson', characters='["Jesus"]'),
 Row(primaryName='Ron Ferguson', characters='["Jesus"]'),
 Row(primaryName='Anthony De Longis', characters='["Jesus"]'),
 Row(primaryName='Marcio Rosario', characters='["Jesus"]'),
 Row(primaryName='Mat Ranillo III', characters='["Jesus Christ"]'),
 Row(primaryName='Andreas Pietschmann', characters='["Jesus"]'),
 Row(primaryName='Matt Christi', characters='["Jesus"]'),
 Row(primaryName='Joerg Stadler', characters='["Christ"]'),
 Row(primaryName='Jesus Amole', characters='["Jesus"]'),
 Row(primaryName='Matt Edens', characters='["Jesus"]'),
 Row(primaryName='Antonino Solmer', characters='["Jesus"]'),
 Row(primaryName='John Paul Jones', characters='["Jesus"]'),
 Row(primaryName='John Paul Jones', characters='["Jesus"]'),
 Row(primaryName='Doug Bauer', characters='["Jesus"]'),
 Row(primaryName='Alain Claessens', characters='["Jesus"]'),
 Row(primaryName='Madison Mason', characters='["Jesus Christ"]'),
 Row(primaryName='Joe Sciammare

In [15]:
alive_actors.show(10)

+-------------------+----------------+
|        primaryName|      characters|
+-------------------+----------------+
|       Ron Ferguson|       ["Jesus"]|
|       Ron Ferguson|       ["Jesus"]|
|  Anthony De Longis|       ["Jesus"]|
|     Marcio Rosario|       ["Jesus"]|
|    Mat Ranillo III|["Jesus Christ"]|
|Andreas Pietschmann|       ["Jesus"]|
|       Matt Christi|       ["Jesus"]|
|      Joerg Stadler|      ["Christ"]|
|        Jesus Amole|       ["Jesus"]|
|         Matt Edens|       ["Jesus"]|
+-------------------+----------------+
only showing top 10 rows

